## Part One: Translate m3diff report output to a more Readable format ##

In [ ]:
import pandas as pd
import re, sys

def extract_fixed_format(line):
    """Extracts values from a fixed-format string similar to Fortran output."""
#    pattern = r"(\w)\s+([\d.E+-]+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([\d.E+-]+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([\d.E+-]+)\s+([\d.E+-]+)"
#    pattern = r"(\w+:\w+)\s+([-+]?\d+\.\d+E[-+]?\d+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([-+]?\d+\.\d+E[-+]?\d+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([-+]?\d+\.\d+E[-+]?\d+)\s+([-+]?\d+\.\d+E[-+]?\d+)"
    pattern = r"(\w+:\w+|\w)\s+([-+]?\d+\.\d+E[-+]?\d+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([-+]?\d+\.\d+E[-+]?\d+)@\(\s*(\d+),\s*(\d+),\s*(\d+)\)\s+([-+]?\d+\.\d+E[-+]?\d+)\s+([-+]?\d+\.\d+E[-+]?\d+)"

    match = re.match(pattern, line)
    if match:
        identifier = match.group(1)
        max_val = float(match.group(2))
        max_loc = (int(match.group(3)), int(match.group(4)), int(match.group(5)))
        min_val = float(match.group(6))
        min_loc = (int(match.group(7)), int(match.group(8)), int(match.group(9)))
        mean_val = float(match.group(10))
        sigm_val = float(match.group(11))

        return {
            "Id": identifier,
            "Max_Val": max_val,
            "Max_Loc": max_loc,
            "Min_Val": min_val,
            "Min_Loc": min_loc,
            "Mean": mean_val,
            "Sigm": sigm_val,
        }
        
    else:
        return None

def parse_data(filename):
    """
    Parses the data from the given file and returns a pandas DataFrame.

    Args:
        filename (str): The name of the file to parse.

    Returns:
        pandas.DataFrame: A DataFrame containing the extracted data.
    """

    data = []#pd.DataFrame(columns=['Date', 'Hour', 'Variable_Name', 'A_Max', 'A_Max_Loc(C,R,L)', 'B_Max', 'B_Max_Loc(C,R,L)', 'Diff_Max(A_Max - B_Max)'])
    with open(filename, 'r') as file:
        lines = file.readlines()

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Check if the line contains "Date and time"
        if "Date and time" in line:
            # Extract date and time
            date_time_str = line.split("Date and time")[1].split("(")[0].strip()
            parts = date_time_str.split(":")
            date_part = parts[0].strip()
            time_part = parts[1].split(" ")[0].strip()

            # Extract variable name from next line
            i += 1
            variable_name = lines[i].split('/')[-1].split()[0].strip()

            # Go to next line which contains headers: MAX        @(  C,  R, L)  Min        @(  C,  R, L)  Mean         Sigma 
            i += 1
            
            extr_dict = extract_fixed_format(lines[i+1].strip()) # Read data line for A
            a_max     = extr_dict['Max_Val']
            a_mxloc   = extr_dict['Max_Loc'] 
            a_min     = extr_dict['Min_Val']
            a_mnloc   = extr_dict['Min_Loc']
            a_mean    = extr_dict['Mean']
            a_sigm    = extr_dict['Sigm']
            
            extr_dict = extract_fixed_format(lines[i+2].strip()) # Read data line for B
            #display(extr_dict)
            b_max     = extr_dict['Max_Val']
            b_mxloc   = extr_dict['Max_Loc'] 
            b_min     = extr_dict['Min_Val']
            b_mnloc   = extr_dict['Min_Loc']
            b_mean    = extr_dict['Mean']
            b_sigm    = extr_dict['Sigm']
            
            extr_dict = extract_fixed_format(lines[i+3].strip()) # Read data line for A - B
            #display(extr_dict)
            dif_max     = extr_dict['Max_Val']
            dif_mxloc   = extr_dict['Max_Loc'] 
            dif_min     = extr_dict['Min_Val']
            dif_mnloc   = extr_dict['Min_Loc']
            dif_mean    = extr_dict['Mean']
            dif_sigm    = extr_dict['Sigm']

            data.append({
                "Date": date_part,
                "Hour": time_part,
                "Variable_Name": variable_name,
                "A_Max": a_max,
                "A_Max_Loc(C,R,L)": a_mxloc,
                "B_Max": b_max,
                "B_Max_Loc(C,R,L)": b_mxloc,
                "Dif_Max(A-B)": dif_max,
                "Dif_Max_Loc": dif_mxloc,
                "A_Min": a_min,
                "A_Min_Loc(C,R,L)": a_mnloc,
                "B_Min": b_min,
                "B_Min_Loc(C,R,L)": b_mnloc,
                "Dif_Min(A-B)": dif_min,
                "Dif_Min_Loc": dif_mnloc,
                "A_Mean": a_mean,
                "B-Mean":b_mean,
                "Dif_Mean":dif_mean,
                "A_Sigm": a_sigm,
                "B-Sigm":b_sigm,
                "Dif_Sigm":dif_sigm,
            })
            i += 3  # Move to the next data block
        else:
            i += 1

    return pd.DataFrame(data)

# Example usage:
flist  = ['METCRO3D','METCRO2D','METDOT3D','GRDCRO2D','LUFRACCRO','GRDDOT2D','SOICRO'] # 'METBDY3D','GRDBDY2D'
dlist  = ['20230401','20230402','20230403']
datadir = "/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck"
for ftype in flist:
    for fdate in dlist:
        rptfile = f"{datadir}/report_m3diff_{ftype}_{fdate}.txt"
        print(rptfile)
        df = parse_data(rptfile)
        #print(df.head())
        #display(df)
        df.to_csv(f"{datadir}/report_m3diff_{ftype}_{fdate}.csv")
        del df

/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO3D_20230401.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO3D_20230402.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO3D_20230403.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO2D_20230401.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO2D_20230402.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METCRO2D_20230403.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METDOT3D_20230401.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METDOT3D_20230402.txt
/nas/longleaf/home/tranhuy/proj/GSA-EMBER/NACC/data/outputs/QA_recheck/report_m3diff_METDOT3D_20230403.txt
/nas/longleaf/home/tranhuy/proj/GSA-E

## Part Two: Comparing variables among two netcdf files across space and time ##

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt 

# Define file paths for different days (update these with actual file paths)
ftype      = ["METCRO3D","METCRO2D","METBDY3D","METDOT3D","GRDCRO2D","GRDDOT2D","GRDBDY2D"]#"SOICRO","LUFRACCRO"]
#ftype      = ["GRDDOT2D","GRDBDY2D"]#,"SOICRO"]
#ftype      = ['METCRO2D']
dates      = ["20230401","20230402","20230403"]

#ftype      = ["GRDCRO2D"]
#dates      = ["20230403"]
#pathA      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/file_check"
#pathB      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/file_check" 
#file_pairs = [(f"{pathA}/{ftype}_{idate}_dupl", f"{pathB}/{ftype}_{idate}") for idate in dates]

#ftype      = ["ACONC"]
#dates      = ["20230401","20230402","20230403"]

thrshd_pct = 0.1 # Threshold of difference in percent %

# Function to process a single day's NetCDF comparison
def process_day(file1, file2, day_index, fltyp):
    print(f"Processing {file1} vs {file2} for Day {day_index + 1}...")

    # Open NetCDF files using dask for lazy loading
    ds1 = xr.open_dataset(file1, engine="netcdf4", chunks="auto")
    ds2 = xr.open_dataset(file2, engine="netcdf4", chunks="auto")

    # Get common variables
    vars1 = set(ds1.variables.keys())
    vars2 = set(ds2.variables.keys())
    common_vars = vars1.intersection(vars2)

    # Store results
    day_results = {}

    # Loop through common variables
    for var in common_vars:
        data1 = ds1[var].data  # Use dask arrays
        data2 = ds2[var].data  # Use dask arrays

        # Ensure the shapes match
        if data1.shape != data2.shape:
            print(f"Shape mismatch for {var} on Day {day_index + 1}: {data1.shape} vs {data2.shape}")
            continue

        # Compute absolute difference
        diff = da.absolute(data1 - data2)

        # Compute reference scale (avoid division by zero)
        #ref = da.maximum(da.absolute(data1), da.absolute(data2))
        ref = (da.absolute(data1) + da.absolute(data2)) / 2
        ref = da.where(ref == 0, -9999, ref)  # Temporary set 0 to -9999 to avoid division by zero

        # Compute relative difference in percentage
        relative_diff = (diff / ref) * 100
        
        # Set relative diff to zero where ref = 0
        relative_diff = da.where(ref == -9999, 0, relative_diff)

        # Count values where difference is < thrshd_pct
        within_threshold = da.sum(relative_diff < thrshd_pct).compute()  # Force computation
        total_values = data1.size
        percentage_within_threshold = (within_threshold / total_values) * 100

        # Store results
        day_results[var] = percentage_within_threshold
        
        # --- Scatter plot generation ---
        aflat = data1.flatten()
        bflat = data2.flatten()
        dflat = da.absolute(aflat - bflat)
        mflat = (aflat + bflat) / 2
        mflat = da.where(mflat == 0, -9999, mflat)  # Avoid division by zero
        
        dflat = (dflat / mflat) * 100
        dflat = da.where(mflat == -9999, 0, dflat)
        mflat = da.where(mflat == -9999, 0, mflat)  # Reset mflat
        
        # Plot
        plt.figure(figsize=(8, 6))
        plt.scatter(mflat, dflat, alpha=0.5, s=10, edgecolors='none')
        plt.xlabel(f"Mean {var} values in {fltyp} ; (A+B)/2")
        plt.ylabel("% Difference abs(A-B)/mean(A,B)")
        plt.title(f"{var} | Day {day_index + 1}")
        plt.grid(True)
        plt.tight_layout()

        # Save plot
        plot_dir = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/nacc_output/scatter_plots"
        plot_path = f"{plot_dir}/scatter_{var}_{fltyp}_day{day_index + 1}.png"
        plt.savefig(plot_path)
        #print(f"Saved scatter plot for {var} → {plot_path}")
        #plt.show()
        plt.close()

    # Close datasets
    ds1.close()
    ds2.close()

    return day_results, day_index


# Run in parallel using ProcessPoolExecutor
for ivar in ftype:
    
    ## For MCIP file
    #pathA      = "/proj/ie/proj/GSA-EMBER/CMAQ_D/36US3/output/EMBER_base/mcip"
    #pathB      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/mcip"
    #file_pairs = [(f"{pathA}/{ivar}_{idate}", f"{pathB}/{ivar}_{idate}") for idate in dates]
    pathA      = "/proj/ie/proj/GSA-EMBER/s3bucket/NACC_MCIP"
    pathB      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs"
    file_pairs = [(f"{pathA}/aqm.t12z.{ivar.lower()}_{idate}.ncf", f"{pathB}/{idate}/aqm.t12z.{ivar.lower()}.{idate}.ncf") for idate in dates]
    #pathA      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/file_check"
    #pathB      = "/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/file_check"
    #file_pairs = [(f"{pathA}/{ivar}_{idate}", f"{pathB}/{ivar}_{idate}_modi") for idate in dates]
    
    ## for ACONC file
    #pathA      = f"/proj/ie/proj/GSA-EMBER/s3bucket/output/CMAQv54_cb6r5_ae7_aq.36US3.35.basecase2023_v2.{ivar}"
    #pathB      = f"/proj/ie/proj/GSA-EMBER/CMAQ_D/36US3/output/EMBER_base/CMAQv54_cb6r5_ae7_aq.36US3.35.EMBER_base.{ivar}" 
    #file_pairs = [(f"{pathA}.{idate}", f"{pathB}.{idate}") for idate in dates]
    
    results = {}
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_day, file1, file2, idx, ivar): idx for idx, (file1, file2) in enumerate(file_pairs)}

        for future in futures:
            day_results, day_index = future.result()
            for var, percent in day_results.items():
                if var not in results:
                    results[var] = [None] * len(file_pairs)  # Initialize with None
                results[var][day_index] = percent

    # Convert results to DataFrame
    df_results = pd.DataFrame.from_dict(results, orient="index", columns=[f"{idate}_{thrshd_pct}%" for idate in dates])

    # Reset index and rename columns
    df_results.reset_index(inplace=True)
    df_results.rename(columns={"index": "Variable"}, inplace=True)

    # Sort DataFrame alphabetically by variable name
    df_results = df_results.sort_values(by="Variable", ascending=True)

    # Print results
    print(df_results)
    df_results.to_csv(f"/proj/ie/proj/GSA-EMBER/NACC/data/outputs/QA_check/nacc_output/QA_Check_{ivar}_{thrshd_pct}pct.csv")

Processing /proj/ie/proj/GSA-EMBER/s3bucket/NACC_MCIP/aqm.t12z.metcro3d_20230402.ncf vs /proj/ie/proj/GSA-EMBER/NACC/data/outputs/20230402/aqm.t12z.metcro3d.20230402.ncf for Day 2...Processing /proj/ie/proj/GSA-EMBER/s3bucket/NACC_MCIP/aqm.t12z.metcro3d_20230403.ncf vs /proj/ie/proj/GSA-EMBER/NACC/data/outputs/20230403/aqm.t12z.metcro3d.20230403.ncf for Day 3...Processing /proj/ie/proj/GSA-EMBER/s3bucket/NACC_MCIP/aqm.t12z.metcro3d_20230401.ncf vs /proj/ie/proj/GSA-EMBER/NACC/data/outputs/20230401/aqm.t12z.metcro3d.20230401.ncf for Day 1...


    Variable  20230401_0.1%  20230402_0.1%  20230403_0.1%
4   CFRAC_3D      99.958177      99.959660      99.958299
2       DENS     100.000000     100.000000     100.000000
15   DENSA_J     100.000000     100.000000     100.000000
10    JACOBF     100.000000     100.000000     100.000000
12    JACOBM     100.000000     100.000000     100.000000
6       PRES     100.000000     100.000000     100.000000
1         PV      99.198959      99.197111   